In [ ]:


!pip install bs4


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
import pandas as pd
import numpy as np
import requests
import itertools
import re

In [ ]:

from bs4 import BeautifulSoup
dict_cities={'BE': ['Bruxelles / Brussel (greater city)', 'Antwerpen', 'Gent', 'Charleroi (greater city)', 'Liège (greater city)', 'Brugge', 'Oostende'], 'BG': ['Sofia', 'Plovdiv', 'Varna', 'Burgas'], 'CZ': ['Praha', 'Brno', 'Ostrava', 'Karlovy Vary'], 'DK': ['København (greater city)', 'Århus', 'Odense', 'Aalborg'], 'DE': ['Berlin', 'Hamburg', 'München', 'Köln', 'Frankfurt am Main', 'Essen', 'Stuttgart', 'Leipzig', 'Dresden', 'Dortmund', 'Düsseldorf', 'Bremen', 'Hannover', 'Nürnberg', 'Bochum', 'Bielefeld', 'Halle an der Saale', 'Magdeburg', 'Wiesbaden', 'Trier', 'Freiburg im Breisgau', 'Regensburg', 'Weimar', 'Erfurt', 'Augsburg', 'Bonn', 'Karlsruhe', 'Mönchengladbach', 'Mainz', 'Kiel', 'Saarbrücken', 'Potsdam', 'Koblenz', 'Rostock', 'Tübingen', 'Flensburg', 'Konstanz', 'Lüneburg', 'Bamberg', 'Stralsund', 'Friedrichshafen', 'Duisburg', 'Mannheim', 'Gelsenkirchen', 'Münster', 'Chemnitz', 'Braunschweig', 'Aachen', 'Krefeld', 'Oberhausen', 'Lübeck', 'Kassel', 'Heidelberg', 'Würzburg', 'Bremerhaven', 'Wuppertal'], 'EE': ['Tallinn', 'Tartu'], 'IE': ['Dublin (greater city)', 'Cork (greater city)', 'Galway (greater city)'], 'EL': ['Athina (greater city)', 'Thessaloniki (greater city)', 'Irakleio', 'Chania'], 'ES': ['Madrid (greater city)', 'Barcelona (greater city)', 'Valencia (greater city)', 'Sevilla (greater city)', 'Zaragoza', 'Málaga', 'Murcia', 'Las Palmas', 'Valladolid', 'Palma de Mallorca', 'Santiago de Compostela', 'Vitoria/Gasteiz', 'Oviedo', 'Pamplona/Iruña (greater city)', 'Toledo', 'Bilbao (greater city)', 'Córdoba', 'Alicante/Alacant', 'Vigo', 'Gijón', 'Santa Cruz de Tenerife (greater city)', 'A Coruña', 'Torrevieja', 'Puerto de Santa María, El', 'Benidorm', 'Fuengirola', 'Torremolinos', 'Granada (greater city)', 'Elche/Elx', 'Cartagena', 'Jerez de la Frontera', 'San Sebastián/Donostia', 'Salamanca', 'Cádiz', 'Marbella', 'Benalmádena', 'Chiclana de la Frontera', 'Eivissa', 'Puerto de la Cruz (greater city)'], 'FR': ['Paris (greater city)', 'Lyon', 'Toulouse', 'Strasbourg', 'Bordeaux', 'Nantes', 'Lille', 'Montpellier', 'Rennes', 'Nancy', 'Reims', 'Orléans', 'Dijon', 'Grenoble', 'Toulon', 'Tours', 'Avignon', 'Bayonne', 'Annecy', 'La Rochelle', 'Fréjus', 'Aix-en-Provence', 'Marseille', 'Nice', 'Rouen', 'Colmar'], 'HR': ['Zagreb', 'Split', 'Pula - Pola', 'Zadar'], 'IT': ['Roma', 'Milano (greater city)', 'Napoli (greater city)', 'Torino', 'Palermo', 'Genova', 'Firenze', 'Bari', 'Bologna', 'Catania', 'Venezia', 'Verona', 'Trieste', 'Cagliari', 'Padova', 'La Spezia', 'Como', 'Pisa', 'Messina', 'Rimini', 'Siracusa', 'Bergamo'], 'CY': ['Lefkosia (greater city)', 'Larnaka (greater city)', 'Lemesos (greater city)'], 'LV': ['Riga'], 'LT': ['Vilnius', 'Kaunas', 'Klaipeda'], 'LU': ['Luxembourg'], 'HU': ['Budapest', 'Debrecen'], 'MT': ['Valletta (greater city)'], 'NL': ['Amsterdam (greater city)', 'Alkmaar (greater city)', 'Almere', 'Apeldoorn', 'Ede (greater city)', 'Eindhoven (greater city)', 's-Gravenhage (greater city)', 'Groningen', 'Haarlem (greater city)', 'Haarlemmermeer', 'Heemskerk (greater city)', 'Leiden (greater city)', 'Maastricht', 'Middelburg (greater city)', 'Rotterdam (greater city)', 'Tilburg', 'Utrecht (greater city)', 'Zaanstad'], 'AT': ['Wien', 'Graz', 'Linz', 'Salzburg', 'Innsbruck'], 'PL': ['Warszawa', 'Lódz', 'Kraków', 'Wroclaw', 'Poznan', 'Gdansk', 'Szczecin', 'Bydgoszcz', 'Lublin', 'Katowice (greater city)', 'Bialystok', 'Torun', 'Czestochowa', 'Radom', 'Gdynia'], 'PT': ['Lisboa (greater city)', 'Porto (greater city)', 'Funchal', 'Ponta Delgada', 'Faro', 'Sintra'], 'RO': ['Bucuresti', 'Cluj-Napoca', 'Timisoara', 'Craiova', 'Braila', 'Oradea', 'Sibiu', 'Constanta', 'Iasi', 'Galati', 'Brasov', 'Ploiesti'], 'SI': ['Ljubljana'], 'SK': ['Bratislava', 'Kosice'], 'FI': ['Helsinki / Helsingfors', 'Tampere / Tammerfors', 'Turku / Åbo', 'Oulu / Uleåborg', 'Espoo / Esbo', 'Vantaa / Vanda', 'Kuopio'], 'SE': ['Stockholm (greater city)', 'Göteborg (greater city)', 'Malmö', 'Umeå', 'Uppsala', 'Lund', 'Borås'], 'IS': ['Reykjavík (greater city)'], 'NO': ['Oslo greater city', 'Stavanger greater city', 'Bergen city', 'Trondheim city'], 'CH': ['Bern (greater city)', 'Zürich (greater city)', 'Genève (greater city)', 'Basel (greater city)', 'Lausanne (greater city)', 'Luzern (greater city)', 'Lugano (greater city)']}

In [ ]:
#EXEMPLE
URL = "https://www.airbnb.fr/s/Casablanca--Grand-Casablanca--Maroc/homes?place_id=ChIJOxGqeEfNpw0R0w8xT9jBBgs&refinement_paths%5B%5D=%2Fhomes&checkin=2022-07-14&checkout=2022-07-17&date_picker_type=calendar&adults=2&children=0&infants=0&pets=0&search_type=AUTOSUGGEST"



In [ ]:
#FONCTIONS POUR RECUPERER LES URLS ET LES PARSER

# Fonction qui récupère l'ensemble des url des différentes pages
def build_urls(main_url, listings_per_page=20, pages_per_location=30):
    url_list = []
    for i in range(pages_per_location):
        offset = listings_per_page * i
        url_pagination = main_url + f'&items_offset={offset}'
        url_list.append(url_pagination)
    
    return url_list


#Fonction qui transforme url en soupe, et récupère les hyperliens pour une page

def html_to_soup(url):
    page=requests.get(url)
    soup=BeautifulSoup(page.content, "html.parser")
    return soup



def extract_info(soupe):
  listings=soupe.find_all(itemprop="itemListElement")
  typelog, links, prices, ratings_rev, superhost =  [], [], [], [], []
  for listing in listings:
      links.append(getListingLink(listing))
      superhost.append(getListingBadges(listing))
      typelog.append(getListingType(listing))
      prices.append(getListingPrice(listing))
      ratings_rev.append(getListingRates(listing))




  dictionary = {"Type&Loc": typelog, "price/nuit": prices, "rating": ratings_rev, "link": links, "superhost":superhost}
  df=pd.DataFrame(dictionary)
  df.loc[:, "amenities"] = df["link"].apply(lambda x: getListingAmenities(html_to_soup(x)))
  df.loc[:, "travellers"] = df["link"].apply(lambda x: getListingHosts(html_to_soup(x)))
  df.loc[:, "Comments"] = df["link"].apply(lambda x: getListingComments(html_to_soup(x)))

  return df


In [ ]:
def main(url):
  pages_url=build_urls(url)
  data=extract_info(html_to_soup(pages_url[0]))
  for pagenumber in range(1, len(pages_url)):
     data=data.append(extract_info(html_to_soup(pages_url[pagenumber])))
  data=data.reset_index()
  #data=DataCleaning_Splitcol(data)
     
  
  return data


In [ ]:
# WEBSCRAPPING


# SUR PAGE PRINCIPALE

def getID(listing):
    '''Retrns the unique ID of a listing'''
    Listing_ID = listing.find("a")["target"]
    return Listing_ID.split("listing_",1)[1]
def getListingLink(listing):
    ''' This function returns the link of the listing'''
    return "http://airbnb.com" + listing.find("a")["href"]
def getListingPrice(listing):
    ''' Returns the price'''
    price_1 = listing.find("span","_tyxjp1").text
    return price_1
def getListingType(listing):
    return listing.find("div","t1jojoys dir dir-ltr").get_text()
def getListingMainfeatures(listing):
  return listing.find("div","f15liw5s s1cjsi4j dir dir-ltr").get_text()
def getListingBadges(listing):
    try:
      listing.find("div","t1mwk1n0 dir dir-ltr").get_text()
      return True
    except: 
      return None

def getListingRates(listing):
  return listing.find("span","ru0q88m dir dir-ltr").get_text()
def getListingAmenities(secondlisting):
  equipements=[]
  for elem in range(len(secondlisting.find_all("div","_19xnuo97"))):
    equipements.append(secondlisting.find_all("div","_19xnuo97")[elem].get_text())
  equipements=", ".join(equipements)
  return equipements
def getListingComments(secondlisting):
  return secondlisting.find("button","_11eqlma4").get_text()
def getListingHosts(secondlisting):
  return secondlisting.find("ol","l7n4lsf dir dir-ltr").get_text()
#Fonction permettant de garder les variables quantitatives brutes 
def Get_decim(string):
  try:
    string=string.replace(",",".")
  except:
    string
  c=""
  for s in re.findall(r'-?\d+\.?\d*', str(string)):
    c=float(s)
  return c



In [ ]:

#DATA CLEANING 

#Fonction qui garde les nombres entiers d'une chaine de caractère


def DataCleaning_Splitcol(df):
  #On split là où c'est nécessaire
  df[['Travellers', 'bedrooms', 'beds',"bathrooms"]]=df["travellers"].str.split(" · ",expand=True)
  df[["log_types","Neighborhood"]]=df["Type&Loc"].str.split("⋅ ",expand=True)
  df["nb_equipements"]=df["amenities"].apply(lambda x: len(x.split(", ")))
  liste_equipements=["wifi","tv","view","washer","parking"]
  #On split les équipements et on crée des variables dummies 
  df=CleanQuantitatives(df)

  for elem in liste_equipements:
    df[elem]=df["amenities"].map(lambda x: x.lower()).str.contains(elem)
  df=df.drop(columns=["Type&Loc","travellers","rating","link","amenities"])
  # On garde les chiffres pour les variables quantitatives, on enlève les signes de ponctuations superflus, tout ca..
  return df

def CleanQuantitatives(df):
  col=["Travellers","bedrooms","beds","bathrooms","Comments","price/nuit"]
  for column in col:
    df[column]=df[column].apply(Get_decim)
  return df

In [ ]:
df

NameError: ignored

In [ ]:

daf=main(URL)


In [ ]:
#Encodage

#On encode les variables quali par one hot encoding
var_quali = ["Neighborhood","Wifi "," Lave-linge"," Arrivée autonome ","parking ","elevator","free parking",] 




In [ ]:
df

,index,Type&Loc,price/nuit,rating,link,superhost,amenities,travellers,Comments
0,0,Appartement ⋅ Casablanca,55 €,"4,92",http://airbnb.com/rooms/plus/15915570?adults=2...,True,,2 guests · 1 bedroom · 1 bed · 1 bath,334 reviews
1,1,Loft ⋅ Casablanca,40 €,Nouveau,http://airbnb.com/rooms/612421874357617781?adu...,None,"Kitchen, Wifi, Dedicated workspace, Pets allow...",2 guests · 1 bedroom · 2 beds · 1 bath,"Casablanca, Casablanca-Settat, Morocco"
2,2,Appartement ⋅ Casablanca,59 €,"5,0",http://airbnb.com/rooms/48066230?adults=2&chec...,True,"City skyline view, Kitchen, Wifi, Dedicated wo...",4 guests · 1 bedroom · 2 beds · 2.5 baths,41 reviews
3,3,Appartement ⋅ Casablanca,58 €,"4,79",http://airbnb.com/rooms/51188986?adults=2&chec...,True,"Kitchen, Wifi, Dedicated workspace, Free parki...",3 guests · 1 bedroom · 1 bed · 1 bath,43 reviews
4,4,Appartement ⋅ Casablanca,43 €,"4,93",http://airbnb.com/rooms/589101024093336777?adu...,None,"City skyline view, Kitchen, Wifi, Dedicated wo...",4 guests · 1 bedroom · 3 beds · 1 bath,15 reviews
...,...,...,...,...,...,...,...,...,...
295,15,Appartement en résidence ⋅ Casablanca,92 €,"4,56",http://airbnb.com/rooms/549208297409529922?adu...,True,"Kitchen, Wifi, Dedicated workspace, Free parki...",5 guests · 2 bedrooms · 2 beds · 2 baths,9 reviews
296,16,Appartement ⋅ Casablanca,110 €,"4,88",http://airbnb.com/rooms/37766282?adults=2&chec...,True,"Kitchen, Wifi, Free parking on premises, TV wi...",6 guests · 2 bedrooms · 2 beds · 1 bath,59 reviews
297,17,Appartement ⋅ Casablanca,77 €,Nouveau,http://airbnb.com/rooms/568020017498291038?adu...,None,"Kitchen, Wifi, Dedicated workspace, Free parki...",2 guests · 1 bedroom · 1 bed · 1 bath,"Casablanca, Casablanca-Settat, Morocco"
298,18,Appartement ⋅ Casablanca,63 €,"4,87",http://airbnb.com/rooms/23778960?adults=2&chec...,None,"Kitchen, Wifi, TV, Elevator, Washer, Dryer, Ai...",2 guests · 1 bedroom · 1 bed · 1 bath,54 reviews


In [ ]:
da=DataCleaning_Splitcol(daf)
da

,index,price/nuit,superhost,Comments,Travellers,bedrooms,beds,bathrooms,log_types,Neighborhood,nb_equipements,wifi,tv,view,washer,parking
0,0,32.0,None,179.0,2.0,1.0,3.0,1.0,Suite,Casablanca,10,True,True,False,True,True
1,1,54.0,True,248.0,2.0,1.0,1.0,1.0,Appartement,Casablanca,10,True,True,False,True,False
2,2,43.0,True,15.0,4.0,1.0,3.0,1.0,Appartement,Casablanca,10,True,True,True,False,True
3,3,59.0,True,41.0,4.0,1.0,2.0,2.5,Appartement,Casablanca,10,True,True,True,True,True
4,4,42.0,True,98.0,2.0,1.0,1.0,1.0,Appartement en résidence,Casablanca,10,True,True,False,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,15,65.0,None,4.0,2.0,1.0,1.0,1.0,Appartement,Maârif,10,True,True,False,False,True
596,16,34.0,None,104.0,6.0,3.0,5.0,1.0,Appartement,Casablanca,10,True,True,True,True,True
597,17,50.0,None,5.0,4.0,1.0,2.0,1.0,Appartement en résidence,Maârif,11,True,True,False,True,True
598,18,81.0,True,58.0,4.0,1.0,1.0,1.5,Appartement,Maârif,10,True,True,False,True,True


In [ ]:
da.to_csv("/content/casablanca.csv")